In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close["close"] = series
    close["close_lag1"] = close["close"].shift(1)
    close["close_lag1_percent"] = (close["close"] - close["close_lag1"]) / close["close_lag1"]
    close["class"] = close["close"]
    for i in range(close.shape[0]):
        if close.loc[i, "close_lag1_percent"] < -0.005:
            close.loc[i, "class"] = 0
        elif close.loc[i, "close_lag1_percent"] < 0:
            close.loc[i, "class"] = 1
        elif close.loc[i, "close_lag1_percent"] < 0.005:
            close.loc[i, "class"] = 2
        else:
            close.loc[i, "class"] = 3
            
    return close["class"].shift(-1).fillna(method="ffill")

In [3]:
data_path: str = "/data/ephemeral/home/BTC/data"
lr_pred = pd.read_csv(os.path.join(data_path, "predicted_values_lasso.csv"))
lgb_pred = pd.read_csv(os.path.join(data_path, "predicted_values_lgb.csv"))
xgb_pred = pd.read_csv(os.path.join(data_path, "predicted_values_xgb.csv"))
close_pred = (lr_pred + lgb_pred + xgb_pred) / 3

target_pred = close_to_class(close_pred)

/tmp/ipykernel_823384/30059925.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return close["class"].shift(-1).fillna(method="ffill")


In [4]:
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
submission_df = submission_df.assign(target = target_pred)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("predicted_values_ensemble.csv", index=False)

In [5]:
submission_df["target"].value_counts()

target
1    1136
2    1117
3     276
0     263
Name: count, dtype: int64